In [40]:
import json
with open('people_detections_with_color_info.json', 'r') as f:
    data = json.load(f)

print(data)

[{'frame_id': 0, 'detections': [{'bbox': [144, 256, 172, 317], 'confidence': 0.7891722321510315, 'track_id': 1, 'class': 'person', 'team': 'red'}, {'bbox': [8, 269, 45, 334], 'confidence': 0.7556441426277161, 'track_id': 2, 'class': 'person', 'team': 'yellow'}, {'bbox': [628, 228, 648, 280], 'confidence': 0.6303842067718506, 'track_id': 3, 'class': 'person', 'team': 'yellow'}, {'bbox': [701, 303, 734, 368], 'confidence': 0.5941866040229797, 'track_id': 4, 'class': 'person', 'team': 'yellow'}, {'bbox': [445, 211, 464, 265], 'confidence': 0.570283830165863, 'track_id': 5, 'class': 'person', 'team': 'unknown'}, {'bbox': [328, 286, 353, 344], 'confidence': 0.3892908990383148, 'track_id': 6, 'class': 'person', 'team': 'yellow'}, {'bbox': [629, 228, 656, 280], 'confidence': 0.36975961923599243, 'track_id': 7, 'class': 'person', 'team': 'yellow'}, {'bbox': [97, 139, 141, 199], 'confidence': 0.33383727073669434, 'track_id': 8, 'class': 'person', 'team': 'yellow'}, {'bbox': [0, 178, 10, 213], '

In [41]:
all_track_ids = []
for frame in data:
    detections = frame['detections']
    for detection in detections:
        all_track_ids.append(detection['track_id'])

print(len(all_track_ids))
print(len(set(all_track_ids)))

all_unique_track_ids = list(set(all_track_ids))

1211
45


In [42]:
def get_overlapping_bboxes(frame):
    detections = frame['detections']
    overlapping_pairs = []
    
    for i, detection1 in enumerate(detections):
        bbox1 = detection1['bbox']
        x1_min, y1_min, x1_max, y1_max = bbox1
        
        for j, detection2 in enumerate(detections[i+1:], i+1):
            bbox2 = detection2['bbox']
            x2_min, y2_min, x2_max, y2_max = bbox2
            
            # Check if bboxes overlap
            if not (x1_max < x2_min or x2_max < x1_min or 
                   y1_max < y2_min or y2_max < y1_min):
                overlapping_pairs.append((detection1['track_id'], detection2['track_id']))
                
    return overlapping_pairs

def get_bbox_size(bbox):
    x1, y1, x2, y2 = bbox
    return (x2 - x1) * (y2 - y1)

In [45]:
track_id_to_colors = {}

for track_id in all_unique_track_ids:
    if track_id not in track_id_to_colors:
        track_id_to_colors[track_id] = []

    for frame in data:
        overlapping_pairs = get_overlapping_bboxes(frame)

        overlapping_track_ids = []
        for i, j in overlapping_pairs:
            overlapping_track_ids.append(i)
            overlapping_track_ids.append(j)
        overlapping_track_ids = list(set(overlapping_track_ids))
        
        detections = frame['detections']
        for detection in detections:
            if detection['track_id'] == track_id:
                if detection['team'] != "unknown":
                    track_id_to_colors[track_id].append(
                        {
                            "team": detection['team'], 
                            "bbox_size": get_bbox_size(detection['bbox']),
                            "has_overlap": track_id in overlapping_track_ids
                        }
                    )


In [47]:
for track_id, colors in track_id_to_colors.items():
    if len(colors) == 0:
        continue
    
    bbox_sizes = [color['bbox_size'] for color in colors]
    max_bbox_size = max(bbox_sizes)

    for color in colors:
        color["confidence"] = 1 - (color["bbox_size"] / max_bbox_size)
        if color["has_overlap"]:
            color["confidence"] = 0

In [48]:
track_id_to_color = {}
for track_id, colors in track_id_to_colors.items():

    if len(colors) == 0:
        track_id_to_color[track_id] = 'unknown'
        continue

    colors_by_confidence = sorted(colors, key=lambda x: x['confidence'], reverse=True)
    # select the top 5
    top_5_colors = colors_by_confidence[:5]

    yellow_count = sum(1 for color in top_5_colors if color['team'] == 'yellow')
    red_count = sum(1 for color in top_5_colors if color['team'] == 'red')
    referee_count = sum(1 for color in top_5_colors if color['team'] == 'referee')

    if yellow_count == 0 and red_count == 0 and referee_count == 0:
        track_id_to_color[track_id] = 'unknown'

    counts = [yellow_count, red_count, referee_count]
    if counts.count(max(counts)) > 1:
        track_id_to_color[track_id] = 'unknown'
        continue

    elif yellow_count == max(counts):
        track_id_to_color[track_id] = 'yellow'
    elif red_count == max(counts):
        track_id_to_color[track_id] = 'red'
    elif referee_count == max(counts):
        track_id_to_color[track_id] = 'referee'

In [49]:
for track_id, color in track_id_to_color.items():
    print(f"Track ID: {track_id}, Color: {color}")

Track ID: 1, Color: red
Track ID: 2, Color: yellow
Track ID: 3, Color: red
Track ID: 4, Color: yellow
Track ID: 5, Color: referee
Track ID: 6, Color: red
Track ID: 7, Color: yellow
Track ID: 8, Color: yellow
Track ID: 9, Color: yellow
Track ID: 134, Color: red
Track ID: 11, Color: yellow
Track ID: 12, Color: red
Track ID: 264, Color: yellow
Track ID: 144, Color: red
Track ID: 274, Color: yellow
Track ID: 275, Color: yellow
Track ID: 277, Color: yellow
Track ID: 23, Color: yellow
Track ID: 259, Color: red
Track ID: 26, Color: yellow
Track ID: 156, Color: referee
Track ID: 47, Color: yellow
Track ID: 175, Color: yellow
Track ID: 181, Color: red
Track ID: 185, Color: red
Track ID: 61, Color: red
Track ID: 69, Color: red
Track ID: 197, Color: unknown
Track ID: 75, Color: red
Track ID: 203, Color: yellow
Track ID: 84, Color: yellow
Track ID: 212, Color: yellow
Track ID: 213, Color: red
Track ID: 215, Color: yellow
Track ID: 93, Color: yellow
Track ID: 104, Color: yellow
Track ID: 105, Color